Cyclingrace results

Cyclingrace is an amateaur road race series.
Each race is a separate event which gives points towards season standing.

In [1]:
import requests
import pandas as pd
import yaml

from collections import defaultdict
from copy import deepcopy

In [2]:
from cr_racing_results import (
    RaceResults,
)

In [3]:
with open('races-config.yaml') as fp:
    race_config = yaml.safe_load(fp)

In [4]:
from pathlib import Path


official_clusters = defaultdict(tuple)

clusters_dir = Path(race_config['clusters'])
for file in list(clusters_dir.iterdir()):
    if file.name.startswith('.'):
        continue
    cluster = file.name.strip('+')
    with open(file) as fp:
        official_clusters[cluster] = official_clusters[cluster] + tuple(fp.read().split('\n'))

In [5]:
race1 = RaceResults.from_config(race_config['races'][0], official_clusters)

In [6]:
race2 = RaceResults.from_config(race_config['races'][0], race1.clusters)
# difference

In [7]:
races = []
cluster_distribution = deepcopy(official_clusters)
for race in race_config['races']:
    races.append(RaceResults.from_config(race, cluster_distribution))
    cluster_distribution = races[-1].clusters
races.sort(key=lambda race: race.order)

In [8]:
cluster_standing = {}

for race in races:
    for cluster, race_points in race.get_race_points().items():
        if cluster in cluster_standing:
            cluster_standing[cluster] = pd.merge(cluster_standing[cluster], race_points, how='outer', on='name')
        else:
            cluster_standing[cluster] = race_points

In [9]:
renamed_columns = {
    "name": "Гонщик",
    "total": "Очки"
}

for cluster in cluster_standing:
    all_results = cluster_standing[cluster].set_index('name')
    all_results['total'] = all_results.sum(axis='columns')
    cluster_standing[cluster] = all_results.sort_values('total', ascending=False)


In [10]:
writer = pd.ExcelWriter('data/current_standing.xlsx', engine = 'xlsxwriter')
for cluster in cluster_standing:
    cluster_standing[cluster].reset_index().to_excel(writer, sheet_name=cluster)

    data = cluster_standing[cluster].reset_index().rename(renamed_columns, axis='columns')
    data.index += 1
    data.to_csv(f'data/cluster_{cluster}.csv')
writer.close()

In [11]:
races[1].group['B'].loc[races[1].group['B'].name.str.startswith('Есипов')]

,bib,name,gender,category,status,rank_abs,result,result_time,team,club
22,136,Есипов Евгений,male,М4 45-49,Q,23.0,02:53:18.09,10398.097,Кластер B. HBFS Plan B,Кластер B. HBFS Plan B


In [12]:
races[1].tt.loc[races[1].tt.name.str.startswith('Есипов')]

,bib,name,gender,category,status,rank_abs,result,result_time,team,club,cluster
58,179,Есипов Евгений,male,М4 45-49,Q,59.0,00:41:16.29,2476.299,HBFS,HBFS,B


In [13]:
list(filter(lambda clust: 'Есипов Е' in races[1].clusters[clust], races[1].clusters))

['B']

# Менялись ли кластеры между гонок

In [16]:
from functools import reduce
all_dfs = reduce(lambda  left,right: pd.merge(left,right, on=['name'], how='outer'), (pd.concat(dfs) for race_name, dfs in all_results.items()))

In [22]:
all_results = defaultdict(list)
for race in race_results['groups']:
    for cluster, results in race.results.items():
        set_tt_cluster(results, official_clusters, 'fixed_cluster')
        set_tt_cluster(results, original_off_clusters, 'official_cluster')
        results['real_cluster'] = cluster
        all_results[race.name].append(results.loc[:, ['name', 'real_cluster']].rename({'real_cluster': race.name}, axis=1))

In [23]:
pd.concat([
    all_dfs.loc[(all_dfs['Дубна'] != all_dfs['Верея']) & ~all_dfs['Дубна'].isna() & ~all_dfs['Верея'].isna()],
    all_dfs.loc[(all_dfs['Садовое кольцо'] != all_dfs['Верея']) & ~all_dfs['Садовое кольцо'].isna() & ~all_dfs['Верея'].isna()],
    all_dfs.loc[(all_dfs['Дубна'] != all_dfs['Верея']) & ~all_dfs['Дубна'].isna() & ~all_dfs['Верея'].isna()]
]).drop_duplicates()

,name,Садовое кольцо,Верея,Дубна
78,Сальников Роман,A,A,B
156,Карпов Антон,A,A,B
370,Дмитриев Андрей,C,C,B
712,Худиев Омар,NaN,B,A
188,Попов Сергей,C,B,B
362,Нечаев Виктор,C,B,B
548,Руез Фабьен,C,A,NaN
